In [1]:
import albumentations as A
import cv2
import data.dataset as ds
import data.utils as utils
import inspect
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pydicom as dicom
import torch
import torch.nn as nn
import torch.nn.functional as F
import yaml

from ast import literal_eval
from scripts.segmentation.train import SegmentationModule
from scripts.scan_classification.train import ScanClassificationModule
from scripts.slice_classification.train import SliceClassificationModule
from slice_classification.model import create_slice_classification_model
from segmentation.model import create_segmentation_model
from sklearn.utils import compute_class_weight
from torch.utils.data import DataLoader, WeightedRandomSampler
from tqdm.auto import tqdm

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.12.1-git20200711.33e2d80-dfsg1-0.6 is an invalid version and will not be supported in a future release
  warnings.warn(


In [5]:
# Segmentation
with open("/home/romainlhardy/kaggle/rsna-abdominal-trauma/configs/segmentation/tf_efficientnetv2_s_128_224_224.yaml", "rb") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
m = SegmentationModule(config["model"])

In [3]:
df = utils.data_split_segmentation("../data/data_split_segmentation.csv")

dataset = ds.SegmentationTrainingDataset(df)

train_dataloader = DataLoader(
    dataset, 
    batch_size=2, 
    sampler=None,
    shuffle=True,
    num_workers=config["num_workers"]
)

In [6]:
batch = next(iter(train_dataloader))
losses = m.step(batch)

RuntimeError: The size of tensor a (6422528) must match the size of tensor b (38535168) at non-singleton dimension 2

In [8]:
y = m.model(batch["image"])
print(y.shape)

torch.Size([4, 6, 128, 224, 224])


In [10]:
from segmentation_models_pytorch.losses import DiceLoss

In [12]:
DiceLoss(mode="multilabel", from_logits=True, smooth=0.1, ignore_index=-100)(torch.randn(4, 6, 128, 224, 224), torch.randint(0, 2, (4, 6, 128, 224, 224)))

tensor(0.4999)

In [ ]:
# Slice classification
with open("/home/romainlhardy/kaggle/rsna-abdominal-trauma/configs/slice_classification/sammed_vit_b_256.yaml", "rb") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
m = SliceClassificationModule(config["model"])

In [ ]:
df = utils.data_split_slice("../data/data_split_slice.csv")

dataset = ds.SliceClassificationTrainingDataset(df, image_size=256)

def collate_fn(batch):
    return {
        "image": torch.stack([b["image"] for b in batch], dim=0),
        "labels": [
            torch.tensor(np.stack([b["labels"][i] for b in batch])) for i in range(7)
        ],
    }

train_dataloader = DataLoader(
    dataset, 
    batch_size=8, 
    sampler=None,
    shuffle=True,
    num_workers=config["num_workers"],
    collate_fn=collate_fn
)

In [ ]:
batch = next(iter(train_dataloader))
clf_losses, seg_loss = m.step(batch, split="train")

In [ ]:
# Scan classification
with open("/home/romainlhardy/kaggle/rsna-abdominal-trauma/configs/scan_classification/lstm_1024_128.yaml", "rb") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
m = ScanClassificationModule(config["model"])

In [ ]:
df = utils.data_split_slice("../data/data_split_scan.csv")

dataset = ds.ScanClassificationTrainingDataset(df, time_dim=1024)

def collate_fn(batch):
    return {
        "features": torch.stack([b["features"] for b in batch], dim=0),
        "labels": [torch.tensor([b["labels"][i] for b in batch]) for i in range(7)],
        "mask": torch.stack([b["mask"] for b in batch], dim=0)
    }

train_dataloader = DataLoader(
    dataset, 
    batch_size=8, 
    sampler=None,
    shuffle=True,
    num_workers=config["num_workers"],
    collate_fn=collate_fn
)

In [ ]:
batch = next(iter(train_dataloader))
clf_losses, seg_loss = m.step(batch, split="train")

In [ ]:
len(batch["labels"])

In [ ]:
probs = [F.softmax(logits, dim=1) for logits in logits_list[1:-1]]
any_injury = (1.0 - torch.cat([p[:, :1] for p in probs], axis=1)).max(dim=1)[0]
torch.log(torch.cat([(1.0 - any_injury)[..., None], any_injury[..., None]], dim=1))

In [ ]:
x = torch.randn(10, 2)

print(F.softmax())

In [ ]:
df = utils.data_split_scan("../data/data_split_scan.csv", "tf_efficientnetv2_s_384_short")

In [ ]:
df.iloc[0].features_path